## Model training and deployment to Azure

#### Installation of accessory libraries

In [1]:
import pycaret 
print(pycaret.__version__)

2.3.10


In [2]:
!pip install azure-storage-blob

In [3]:
!pip install git+https://github.com/amjadraza/pycaret.git@feature/gcp_azure_np_docs

### Model Development

In [4]:
from pycaret.datasets import get_data
dataset = get_data('diamond')

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
0,1.10,Ideal,H,SI1,VG,EX,GIA,5169
1,0.83,Ideal,H,VS1,ID,ID,AGSL,3470
2,0.85,Ideal,H,SI1,EX,EX,GIA,3183
3,0.91,Ideal,E,SI1,VG,VG,GIA,4370
4,0.83,Ideal,G,SI1,EX,EX,GIA,3171


In [5]:
data = dataset.sample(frac=0.95, random_state=786).reset_index(drop=True)
data_unseen = dataset.drop(data.index).reset_index(drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (5700, 8)
Unseen Data For Predictions: (300, 8)


In [6]:
from pycaret.regression import *
exp_reg101 = setup(data = data, target = 'Price', session_id=123,
                  silent = True)

,Description,Value
0,session_id,123
1,Target,Price
2,Original Data,"(5700, 8)"
3,Missing Values,False
4,Numeric Features,1
5,Categorical Features,6
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(3989, 28)"


In [7]:
lightgbm = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,762.6653,3673803.8234,1916.7169,0.9701,0.0728,0.0523
1,955.5267,7425168.5522,2724.9162,0.9400,0.0850,0.0588
2,852.9126,5146188.1541,2268.5211,0.9579,0.0834,0.0610
3,699.4826,1730518.0345,1315.4916,0.9811,0.0766,0.0557
4,633.4070,1505126.5084,1226.8360,0.9852,0.0681,0.0523
5,758.1128,2698170.1978,1642.6108,0.9727,0.0784,0.0581
6,847.3121,6219282.8008,2493.8490,0.9394,0.0811,0.0583
7,729.4342,2328849.0908,1526.0567,0.9728,0.0765,0.0560
8,837.5604,2614949.9435,1617.0807,0.9738,0.0824,0.0622


In [8]:
tuned_lightgbm = tune_model(lightgbm)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,897.4104,4319083.4486,2078.2405,0.9649,0.1058,0.0735
1,979.6476,6389276.0120,2527.7017,0.9484,0.1008,0.0716
2,1021.0107,6827802.9060,2613.0065,0.9441,0.1134,0.0820
3,769.5148,1730852.8293,1315.6188,0.9811,0.0972,0.0692
4,888.5994,2700570.6341,1643.3413,0.9735,0.1067,0.0772
5,893.8512,2851437.5824,1688.6200,0.9712,0.1264,0.0816
6,917.3538,6758753.0735,2599.7602,0.9342,0.1057,0.0713
7,830.5923,2810889.0461,1676.5706,0.9672,0.0934,0.0691
8,872.8899,2521854.2346,1588.0347,0.9747,0.0913,0.0702


In [9]:
predict_model(tuned_lightgbm );

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,815.2298,2663924.4724,1632.1533,0.9739,0.0972,0.0690


### Finalize the model

In [10]:
final_lightgbm  = finalize_model(tuned_lightgbm )

In [11]:
print(final_lightgbm)

LGBMRegressor(bagging_fraction=0.6, bagging_freq=2, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.4,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=41, min_child_weight=0.001, min_split_gain=0.9,
              n_estimators=260, n_jobs=-1, num_leaves=70, objective=None,
              random_state=123, reg_alpha=2, reg_lambda=3, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


In [12]:
predict_model(final_lightgbm);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,664.3089,1701075.3691,1304.2528,0.9833,0.0775,0.0556


# Deploy the model to Microsoft Azure

This is the code to implement the model in Microsoft Azure using the `pycaret` features.

In [13]:
def create_container(container_name):

  # Create the container
    container_client = blob_service_client.create_container(container_name)

    return container_client

def upload_blob(container_name, source_file_name, destination_blob_name):

  # Create a blob client using the local file name as the name for the blob
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=destination_blob_name)

    print("\nUploading to Azure Storage as blob:\n\t" + source_file_name)

  # Upload the created file
    with open(source_file_name, "rb") as data:
        blob_client.upload_blob(data)

def download_blob(container_name, source_blob_name, destination_file_name):
  # Download the blob to a local file
    print("\nDownloading blob to \n\t" + destination_file_name)

  # Create a blob client using the local file name as the name for the blob
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=source_blob_name)

    if destination_file_name is not None: 
        with open(destination_file_name, "wb") as download_file:
            download_file.write(blob_client.download_blob().readall())

        print(
            "Blob {} downloaded to {}.".format(
                source_blob_name, destination_file_name
            )
        )

In [14]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

#### Here you should put the access token (Connection String)

In [18]:
## Enter connection string when running in google colab
connect_str = 'DefaultEndpointsProtocol=https;AccountName=machinelearninginsurance;AccountKey=VvrtwyVNMZg7rnks4Bfg34gQDBKgRYrbUWI/QzLt+61x+hfW7Q+GPah8NOa1dhVpA26sIhGlwe/8+AStfRIdGw==;EndpointSuffix=core.windows.net' 
print(connect_str)

DefaultEndpointsProtocol=https;AccountName=machinelearninginsurance;AccountKey=VvrtwyVNMZg7rnks4Bfg34gQDBKgRYrbUWI/QzLt+61x+hfW7Q+GPah8NOa1dhVpA26sIhGlwe/8+AStfRIdGw==;EndpointSuffix=core.windows.net


In [19]:
# Create the BlobServiceClient object which will be used to create a container client
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

##### Create the container in the Blob Storage

In [20]:
# Create a unique name for the container
container_name = "machinelearninginsurance"
container_client = create_container(container_name)
model_dir = 'modelo'

In [21]:
# Save Model Local and upload to Azure
model_name_azure = 'lightgbm-reg-101'
save_model(final_lightgbm, 
           model_name= model_name_azure, 
           verbose=False)

model_src = model_name_azure +'.pkl'
model_dst = str(model_name_azure)+'.pkl'

##### Upload the model

In [22]:
upload_blob(container_name, model_src, model_dst)


Uploading to Azure Storage as blob:
	lightgbm-reg-101.pkl
